In [6]:
import numpy as np
import pandas as pd
import gc

In [7]:
def changeDType(df,flag=False):
    
    if(flag):
        numericDtype = ['int32','int64','float64','float32']
    
    for i in df.columns:
        if (df[i].dtype == 'int64' or df[i].dtype == 'int32'):
            df[i] = pd.to_numeric(df[i],downcast='integer')
        
        if (df[i].dtype == 'float64' or df[i].dtype == 'float32'):
            df[i] = pd.to_numeric(df[i],downcast='float')

In [8]:
dfTrain = pd.read_csv('../input/proprocessedtrain/preprocessesTrain.csv')
dfTest = pd.read_csv('../input/testdata/preprocessedTest.csv')

In [9]:
changeDType(dfTrain)
changeDType(dfTest)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [ ]:
# n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1,
# min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0,
# bootstrap=True, oob_score=False, n_jobs=None,
# random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None

In [23]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [30]:

def tuneHyperParams(df):
    param = {
        'n_estimators' : [30,40,50,60,75,95,110,140,150,170,200,220,250,265,270,275],
        'criterion' : ['gini','entropy'],
        'max_depth' : [10,15,20,35,45,60,75,95,105,None],
        'min_samples_split' : [0.2,0.3,0.4,0.55,0.65,0.8,0.92,1.0],
        'max_features' : [0.2,0.25,0.3,0.35,0.45,0.5,0.6,0.7,None],
        'bootstrap':[True,False],
        'n_jobs' : [-1],
        'max_samples' : [0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,None]
    }
    
    rf = RandomForestClassifier(n_jobs=-1)
    
    rf_rand = RandomizedSearchCV(estimator=rf,param_distributions=param,verbose=1,n_jobs=-1,n_iter=20,)
#     rf_grid = (estimator=rf,param_grid=param,cv=5,verbose=1,n_jobs=-1)
    
    train,test = train_test_split(df,test_size=0.3)
    
    x = train.drop(['isFraud'],axis=1)
    y = train['isFraud']
    
    rf_rand.fit(x,y)
    
    bestParams = rf_rand.best_params_
    
    print(bestParams)
    print(rf_rand.best_score_)

In [31]:
tunedParams = tuneHyperParams(dfTrain)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 380, in fit
    "`max_sample` cannot be set if `bootstrap=False`. "
ValueError: `max_sample` cannot be set if `bootstrap=False`. Either switch to `bootstrap=True` or set `max_sample=None`.

  warnings.warn(some_fits_failed_message, Fit

{'n_jobs': -1, 'n_estimators': 170, 'min_samples_split': 0.2, 'max_samples': 0.6, 'max_features': 0.45, 'max_depth': 20, 'criterion': 'gini', 'bootstrap': True}
0.9696253411018512


In [32]:
def randomForest(df,isTest = False,model=None):
    
    train,test = train_test_split(df,test_size=0.25)
    
    rf = RandomForestClassifier(n_estimators=170,min_samples_split=0.2,bootstrap=True,max_depth=20,criterion="gini",n_jobs=-1,verbose=1,max_samples=0.6,max_features=0.45)
    
    x = train.drop(['isFraud'],axis=1)
    y = train['isFraud']
   
    x = x-x.min()/x.max()-x.min()
    
    rf.fit(x,y)
    
    xt = test.drop(['isFraud'],axis=1)
    yt = test['isFraud']
    
    print("Accuracy : ",rf.score(xt,yt))
    
    prob = rf.predict_proba(xt)[:,1]
    
    auc = metrics.roc_auc_score(yt,prob)
    print("AUC : ",auc)
    return [rf,auc]

In [33]:
def predict(df,model):
    
    x = df
    x = x - x.min()/x.max()-x.min()
    
    y_pred = model.predict(x)
    
    result = pd.DataFrame(y_pred)
    
    result.columns = ['isFraud']
    
    result.to_csv('./result_rf_wo_tunning.csv')
    print("Result saved")
    return
    
    

In [34]:
model = randomForest(dfTrain)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done 170 out of 170 | elapsed:  1.5min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 170 out of 170 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s


Accuracy :  0.9698812596799175
AUC :  0.8285532553706703


[Parallel(n_jobs=4)]: Done 170 out of 170 | elapsed:    0.4s finished


In [35]:
predict(dfTest,model[0])

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 170 out of 170 | elapsed:    0.6s finished


Result saved
